Кобзарь О.С. Хабибуллин Р.А. 31.07.2019

# Подготовка данных для расчета

* Вырезка куска данных для требуемого промежутка времени, используя подготовленные исходные данные
* Обработка малого куска данных в требуемом промежутке времени
* Тестирование и просмотр полученных данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

from datetime import date

# 1. Извлечение исходных данных на определенный промежуток времени

Сперва загрузим предварительно обработанные данные со станции управления (обработка производилась для разнесения параметров по столбцам с помощью ноутбука `CS_big_data_viewer_and_editor`

In [ ]:
test = pd.read_csv("edited_kobzar2.csv", index_col = "2")

In [ ]:
test.info()

In [ ]:
test.head()

Определим промежуток времени с которым будем работать для извлечения данных из большого DataFrame 

In [ ]:
year_start = 2019
month_start = 2
day_start = 1
hour_start = 0

year_end = 2019
month_end = 2
day_end = 28
hour_end = 0
date_from = pd.to_datetime(pd.DataFrame({'year': [year_start, year_end],
                           'month': [month_start, month_end],
                           'day': [day_start, day_end],
                            'hour': [hour_start, hour_end]}))
date_start = date_from.iloc[0]
print("Дата начала периода для тщательного анализа: " + str(date_start))
date_end = date_from.iloc[1]
print("Дата конца периода для тщательного анализа: " + str(date_end))

Вырежем кусок данных и запишем его

In [ ]:
test.index = pd.to_datetime(test.index)
small_test = test[(test.index >= date_start) & (test.index <= date_end)]
small_test.to_csv('small_part_to_analyze.csv')
small_test.head()

# 2. Тщательная обработка интересующего малого куска данных

Загрузим небольшой обработанный кусок данных и доготовим его до правильной кондиции и использования в расчетах с помощью ресемлирования и прочее

In [ ]:
loaded_file = pd.read_csv('small_part_to_analyze.csv', index_col = '2')
loaded_file.index = pd.to_datetime(loaded_file.index)
loaded_file.head()

In [ ]:
loaded_file.info()

Получить номера имен столбцов для удобного обращения к данным

In [ ]:
for i, name in enumerate(loaded_file.columns):
    print(str(i) + '  -  ' + name)

Проверим некоторые столбцы на заполненность

In [ ]:
loaded_file[loaded_file[loaded_file.columns[11]].notna() == True][loaded_file.columns[11]]

In [ ]:
loaded_file[loaded_file[loaded_file.columns[12]].notna() == True][loaded_file.columns[12]]

In [ ]:
loaded_file[loaded_file[loaded_file.columns[20]].notna() == True][loaded_file.columns[20]]

In [ ]:
loaded_file[loaded_file[loaded_file.columns[23]].notna() == True][loaded_file.columns[23]]

Получим список параметров (названий столбцов), которые представляют интерес для расчета

In [ ]:
needed_param_numbers = [0,11,14,19,22,24,25,30,33,34,35,36,38,40,51]
needed_param_list = []
for i in needed_param_numbers:
    needed_param_list.append(loaded_file.columns[i])
for i, name in enumerate(needed_param_list):
    print(str(i) + '  -  ' + name)

Удалим столбы, не представляющие интерес для расчета

In [ ]:
for i in loaded_file.columns:
    if i not in needed_param_list:
        del loaded_file[i]

Произведем ресемплирование, запишем в файл и посмотрим на результат

In [ ]:
check_resample = loaded_file.resample('1d').mean()
check_resample.to_csv("resampled_small_data.csv")
check_resample.head()

# 3. Проверка подготовленных итоговых данных

Просмотр и построение графиков

In [ ]:
check_resample = pd.read_csv("resampled_small_data.csv", index_col = '2')
check_resample.head()
#check_resample.iloc[-2]

In [ ]:
trace1_column_name = needed_param_list[10]
trace1 = go.Scattergl(
    x = check_resample.index,
    y = check_resample[trace1_column_name],
    name = trace1_column_name,
    mode = 'markers'
)

trace2_column_name = needed_param_list[11]
trace2 = go.Scattergl(
    x = check_resample.index,
    y = check_resample[trace2_column_name],
    name = trace2_column_name,
    mode = 'markers'
)
trace3_column_name = needed_param_list[13]
trace3 = go.Scattergl(
    x = check_resample.index,
    y = check_resample[trace3_column_name],
    name = trace3_column_name,
    mode = 'markers'
)


data = [trace1, trace2, trace3]


layout = dict(title = 'Просмторе обработанных данных, которые идут в расчет'
             )

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')